# **Camino de Santiago - Data Collector**

### **Introduction**

Welcome to the "Camino de Santiago Data Collector" Notebook. This interactive tool is designed to collect statistical data on pilgrims traveling the Camino de Santiago. By leveraging an API provided by the Cathedral of Santiago, this notebook streamlines the process of data collection, processing, and transformation to gain insights into the pilgrimages.

### **Objective**

The primary goal of this notebook is to provide a structured and automated method for data collection, enabling analysts and enthusiasts to discover trends and patterns in pilgrim activities. The data gathered can be utilized for reporting, academic research, or to create dashboards for visual representations.

### **Structure of the Notebook**

The notebook is organized into several sections, each dedicated to a specific task within the data processing workflow:

- **Load Libraries:** Import all necessary libraries for data handling and define parameters.
- **Create Dataframes:** Build a data structure to store the retrieved information.
- **Define Functions:** Develop functions for data retrieval and preprocessing.
- **Data Transformation:** Transform and clean the data for analysis.
- **Save the Data:** Store the collected data in an Excel file and multiple CSV files.

### **How to Use**

After defining the parameters, the notebook should run from top to bottom without any further interaction.

### **API End of Service**

Unfortunately, the API ceased functioning in May 2023. Therefore, the current data cannot be fetched by this notebook.

We extend our gratitude to the [Office of the Pilgrim in Santiago](https://www.notion.so/Hubvisory-da7f22856fb9482d883af0a55d3c2cd5?pvs=21) for providing the data and to Kaggle user [Guillermo Ibarra](https://www.kaggle.com/code/guillermoibarra/camino-de-santiago-async-restful-extract)  for the foundational work that inspired this notebook.

Best of luck with your analysis of the Camino de Santiago pilgrim data, and a heartfelt "Buen Camino" to all the pilgrims on their journey!

#### __Step 1:__ Load libraries

In [1]:
import time                       # To work with time objects
import json                       # To process returned JSON data
import pandas as pd               # To work with dataframes
import asyncio                    # To make async requests
import httpx                      # To make requests to Cathedral URL (alternative to requests with async support)
import datetime                   # To work with datetime objects
import openpyxl                   # To save to Excel files
import os                         # To work with files and directories


### Set Some Parameters


In [60]:
# URL of the Cathedral API in Santiago de Compostela
base_url = 'https://catedral.df-server.info/ws/wsCatedral.asmx'

# Define the start and end dates for the data query
data_start_date = datetime.date(2004, 1, 1)  # Start date is January 2004
data_end_date = datetime.date.today()  # Use today's date as the end date

# Output path for csv files
save_path_csv = f"/home/david/code/projects/CaminoStats/data/csv_files/Caminostats_{data_start_date}_{data_end_date}/"

# Output path and name for Excel file
save_path_xl = "/home/david/code/projects/CaminoStats/data/exel_files/"
filename_xlsx = f"Caminostats_{data_start_date}_{data_end_date}.xlsx"


#### __Step 2:__ Creating a dict of dataframes to hold the data

In [3]:
# Dataframes to fill

tablelist = [
    'Autonomias',
    'Caminos',
    'Continentes',
    'Edades',
    'Grupos',
    'Medios',
    'Motivos',
    'Paises',
    'Procedencias',
    'Sexos'
]

# Define the common columns for the dataframes
common_columns = ['id', 'Anho', 'Mes', 'Nombre', 'Total', 'Porcentaje']

# Define a dictionary to hold all the DataFrames
dataframes = {
    'Totals': pd.DataFrame(columns=['Anho', 'Mes', 'Identificador', 'TotalRegistros']),
    # Other DataFrame names can follow the same naming convention as 'tablelist'
}

# Create a DataFrame for each item in tablelist and add it to the dictionary
for table_name in tablelist:  # Skipping the first three items as they are already accounted for
    dataframes[table_name] = pd.DataFrame(columns=common_columns)


#### __Step 3:__ Define functions that calculate the valid range of dates that may be retrieved from the server

In [4]:
# This function calculates the range of valid months between two dates
def range_of_months(start_date, end_date):
    months = []
    # Loop from start to end month, converting the month number back to year and month
    for month_num in range(start_date.year * 12 + start_date.month - 1, end_date.year * 12 + end_date.month):
        year, month = divmod(month_num, 12)
        months.append([year, month + 1])  # Add 1 because months are 1-indexed
    return months


In [5]:
# Generate a list of year and month pairs between the start and end dates
year_months = range_of_months(data_start_date, data_end_date)


#### __Step 4:__ Define functions that concatenate newly retrieved data to previously retrieved data

In [6]:
# Function to add prefixed columns for ID, year, and month to a DataFrame
def add_prefixed_columns(idx, yr, mnth, df):
    df['id'] = idx
    df['Anho'] = yr
    df['Mes'] = mnth
    return df


In [7]:
# Function to add new data to a running totals DataFrame
def add_to_dataframe(df_running_totals, yr, mnth, df_to_add):
    idx = f"{yr}{str(mnth).zfill(2)}"
    add_prefixed_columns(idx, yr, mnth, df_to_add)
    return pd.concat([df_running_totals, df_to_add], ignore_index=True)


#### __Step 5:__ Defines function that retrieves, splits and converts json data to the various dataframes

In [8]:
# Async function that requests data from the Cathedral's server
# waits for response from server before it continues with requests to avoid empty responses (a "denial of service" defense mechanism? )

async def get_data(month, year, dataframes):
    url = f"{base_url}/ObtenerEstadisticasMes?eAnho={year}&eMes={month}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        if response.status_code == 200:
            json_data_chunk = response.json()
            if json_data_chunk['Caminos'] == []:
                print(f"No data for month: {month}, year: {year}")
            else:
                dataframes['Totals'] = add_to_dataframe(dataframes['Totals'], year, month, pd.DataFrame({'TotalRegistros': [json_data_chunk['TotalRegistros']],'Identificador': [json_data_chunk['Identificador']]}))
                keylist = ['TotalRegistros','Identificador','Anho','Mes']
                for key in json_data_chunk.keys():
                    #print(key)
                    if key in tablelist:
                        #print(f"Adding data for {key}")
                        dataframes[key] = add_to_dataframe(dataframes[key], year, month, pd.DataFrame(json_data_chunk[key]))
                        #print(dataframes[key].tail())
                print(f"Got data for month: {month}, year: {year}")
        else:
            print(f"Failed to get data for month: {month}, year: {year}, Status Code: {response.status_code}")
    return dataframes


In [9]:
# Generate a list of year and month pairs between the start and end dates
# If already data exists, start from the last month in the data
if len(dataframes['Totals'])>0:
    data_start_date = datetime.date(int(dataframes['Totals'].tail(1)['id'].values[0][:4]), int(dataframes['Totals'].tail(1)['id'].values[0][4:])+1, 1)

year_months = range_of_months(data_start_date, data_end_date)

# Loop through each year and month pair to get data
for year, month in year_months:
    result = await get_data(month, year, dataframes)
    if not result:
        print('Breaking loop due to an error')
        break


Got data for month: 1, year: 2004
Got data for month: 2, year: 2004
Got data for month: 3, year: 2004
Got data for month: 4, year: 2004
Got data for month: 5, year: 2004
Got data for month: 6, year: 2004
Got data for month: 7, year: 2004
Got data for month: 8, year: 2004
Got data for month: 9, year: 2004
Got data for month: 10, year: 2004
Got data for month: 11, year: 2004
Got data for month: 12, year: 2004
Got data for month: 1, year: 2005
Got data for month: 2, year: 2005
Got data for month: 3, year: 2005
Got data for month: 4, year: 2005
Got data for month: 5, year: 2005
Got data for month: 6, year: 2005
Got data for month: 7, year: 2005
Got data for month: 8, year: 2005
Got data for month: 9, year: 2005
Got data for month: 10, year: 2005
Got data for month: 11, year: 2005
Got data for month: 12, year: 2005
Got data for month: 1, year: 2006
Got data for month: 2, year: 2006
Got data for month: 3, year: 2006
Got data for month: 4, year: 2006
Got data for month: 5, year: 2006
Got data

In [10]:
dataframe_backup = dataframes.copy()


In [32]:
#dataframes = dataframe_backup.copy()


In [54]:
with pd.ExcelWriter(f"{save_path_xl}backup_{filename}", engine="openpyxl") as writer:
    for key in dataframe_backup.keys():
        dataframe_backup[key].to_excel(writer, sheet_name=key)


ValueError: Invalid extension for engine 'openpyxl': 'csv'

#### __Step 6:__ Rename and reorganize the dataframes

In [33]:
# Dictionary mapping current column names to the new names in proper case
column_names = {
    "id": "ID",
    "Anho": "Year",
    "Mes": "Month",
    "Nombre": "Description",
    "Total": "Total",
    "Porcentaje": "Percentage",
    "TotalRegistros": "TotalRecords",
    "Identificador": "Identifier"
}


In [34]:
# Function to rename dataframe columns according to a given mapping
def rename_columns(df, new_column_names):
    return df.rename(columns=new_column_names)


In [35]:
# Rename columns in all dataframes
for key in dataframes.keys():
    dataframes[key] = rename_columns(dataframes[key], column_names)


__edit Totals Table__

In [36]:
dataframes['Totals'] = dataframes['Totals'].set_index(['ID', 'Year', 'Month']).rename(columns={'TotalRecords': 'Total'})


In [37]:
dataframes['Totals']


,,,Identifier,Total
ID,Year,Month,,
200401,2004,1,200401,643
200402,2004,2,200402,1401
200403,2004,3,200403,3105
200404,2004,4,200404,15554
200405,2004,5,200405,16860
...,...,...,...,...
202211,2022,11,202211,11283
202212,2022,12,202212,4683
202301,2023,1,202301,2028


#### __Step 8:__ Converts tables format from __"long"__ to __"wide"__, in this step all nulls are converted to zero because these are numeric counts




__For df Autonomias__

In [38]:
# Create the pivot table directly with summation
autonomous_coms_wide_df = pd.pivot_table(           # Create a pivot table
                dataframes['Autonomias'],           # Use the 'Autonomias' dataframe
                values='Total',                     # Use the 'Total' column for the values
                index=['ID', 'Year', 'Month'],      # Use the 'ID', 'Year', and 'Month' columns for the index
                columns='Description',              # Use the 'Description' column for the columns
                aggfunc='sum',                      # Aggregation function is required but will not change the output if no duplicates
                fill_value=0                        # Replace NaN values with 0 directly
                )

# pop df Autonomias
dataframes.pop('Autonomias', None)

# Reorder columns if necessary and add the 'Total' column
cols = autonomous_coms_wide_df.columns.tolist()

# This line might be redundant if the column order is already correct.
autonomous_coms_wide_df = autonomous_coms_wide_df[cols]
autonomous_coms_wide_df['Total'] = autonomous_coms_wide_df.sum(axis=1)

# Update the `dataframes` dictionary
dataframes['Autonomous Communities'] = autonomous_coms_wide_df
del autonomous_coms_wide_df


__For df Caminos__

In [39]:
# Directly create the pivot table with summation
routes_wide_df = pd.pivot_table(
    dataframes['Caminos'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',
    fill_value=0
)

# pop df Caminos
dataframes.pop('Caminos', None)

# Calculate the 'Total' column as the sum of the rows
routes_wide_df['Total'] = routes_wide_df.sum(axis=1)

# Translate columns to English
routes_translation = {
    'Frances-Camino de': 'Camino Francés',
    'Via de la Plata': 'Vía de la Plata',
    'Primitivo-Camino': 'Camino Primitivo',
    'Portugues-Camino': 'Camino Portugués',
    'Norte-Camino de': 'Camino del Norte',
    'Ingles-Camino': 'Camino Inglés',
    'Costa Camino Portugues': 'Camino Portugués de la Costa',
    'Otros caminos': 'Other Ways',
    'Muxia-Finisterre': 'Camino de Finisterre y Muxía',
    'Invierno de Camino': 'Winter Way (Camino de Invierno)',
    'Resto': 'Others',
    'Geira e Arrieiros': 'Geira and Arrieiros Way (Camino de Geira e Arrieiros)',
    'Muros - Noia Camino': 'Muros-Noia Way (Camino de Muros - Noia)',
    'Miñoto Ribeiro': 'Miñoto Ribeiro Way (Camino de Miñoto Ribeiro)',
    'Camino del Barbanza': 'Barbanza Way (Camino del Barbanza)',
    'Camino del Mar': 'Way of the Sea (Camino del Mar)',
    'Camino olvidado': 'Forgotten Way (Camino Olvidado)',
    'San Salvador': 'San Salvador Way (Camino de San Salvador)',
    'Vía Céltica': 'Celtic Way (Vía Céltica)',
    'San Rosendo': 'San Rosendo Way (Camino de San Rosendo)',
    'Sin establecer': 'Not Established'
}
routes_wide_df.rename(columns=routes_translation, inplace=True)

# Update the `dataframes` dictionary and delete the temporary dataframe
dataframes['Routes'] = routes_wide_df
del routes_wide_df


__For df Continents__

In [40]:
# Create the pivot table directly with summation
continents_wide_df = pd.pivot_table(
    dataframes['Continentes'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',
    fill_value=0
)

# Calculate the 'Total' column as the sum of the rows
continents_wide_df['Total'] = continents_wide_df.sum(axis=1)

# Translate columns to English
column_mapping = {
    'Europa': 'Europe',
    'América del Norte': 'North America',
    'Oceanía': 'Oceania',
    'América del Sur': 'South America',
    'Asia': 'Asia',
    'África': 'Africa',
    'Resto': 'Other',
    'Sin establecer': 'Not specified'
}


continents_wide_df.rename(columns=column_mapping, inplace=True)


# Update the `dataframes` dictionary and delete the temporary dataframe
dataframes['Continentes'] = continents_wide_df


# It seems that there is 2 Africas, so we need to sum them
# Calculate the sum for Africa and add it as a new column
africa = dataframes['Continentes']['Africa'].sum(axis=1)
dataframes['Continentes'].drop('Africa', axis=1, inplace=True)

dataframes['Continentes']['Africa'] = africa

# Reorder and select only the required columns
columns_order = ['Europe', 'North America', 'South America', 'Asia', 'Africa', 'Oceania', 'Other', 'Not specified', 'Total']
dataframes['Continentes'] = dataframes['Continentes'][columns_order]


del continents_wide_df


__For df Edades__

In [41]:
# Pivot without groupby if there are no duplicate combinations
ages_wide_df = pd.pivot_table(
    dataframes['Edades'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',
    fill_value=0
)

# pop df Edades
dataframes.pop('Edades', None)

# Add the 'Total' column as the sum of the rows
ages_wide_df['Total'] = ages_wide_df.sum(axis=1)

# Combine 'Sin establecer' and 'Resto' columns into 'Not defined'
columns_to_combine = ['Sin establecer', 'Resto']
ages_wide_df['Not defined'] = 0

for column in columns_to_combine:
    if column in ages_wide_df.columns:
        ages_wide_df['Not defined'] += ages_wide_df.pop(column)  # Use pop to add and remove the column

# Ensure the columns are in the desired order
order = ['30-60', '<30', '>60', 'Not defined', 'Total']

# Reindex columns based on the 'order' list, missing columns will be filled with NaN, then replace NaN with 0
ages_wide_df = ages_wide_df.reindex(order, axis=1, fill_value=0)

# Update the `dataframes` dictionary and delete the temporary dataframe
dataframes['Ages'] = ages_wide_df


__For df Grupos__

In [42]:
# Pivot the DataFrame to wide format
groups_wide_df = pd.pivot_table(
    dataframes['Grupos'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',  # sum is used but has no effect if there are no duplicates
    fill_value=0    # Replace NaN with 0
)
# pop df Grupos
dataframes.pop('Grupos', None)

# Add the 'Total' column as the sum of the other columns
groups_wide_df['Total'] = groups_wide_df.sum(axis=1)

# Translate columns to English
groups_column_mapping = {
    'Estudiantes': 'Students',
    'Obreros': 'Workers',
    'Liberales': 'Freelancers',
    'Empleados': 'Employees',
    'Profesores': 'Teachers',
    'Jubilados': 'Retirees',
    'Tecnicos': 'Technicians',
    'Funcionarios': 'Civil Servants',
    'Parados': 'Unemployed',
    'Amas de Casa': 'Housewives',
    'Resto': 'Others',
    'Agricultores': 'Farmers',
    'Artistas': 'Artists',
    'Deportistas': 'Athletes',
    'Directivos': 'Executives',
    'Marinos': 'Sailors',
    'Religiosas/os': 'Religious',
    'Sacerdotes': 'Priests',
    'Sin establecer': 'Not specified'
}
groups_wide_df.rename(columns=groups_column_mapping, inplace=True)

dataframes['Groups'] = groups_wide_df
del groups_wide_df


__For df Medios__

In [43]:
# Directly create the pivot table with summation
transportation_wide_df = pd.pivot_table(
    dataframes['Medios'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',  # Summing the total values
    fill_value=0    # Replacing NaN values with 0 directly
)

# pop df Medios
dataframes.pop('Medios', None)

# Calculate the 'Total' column as the sum of the rows
transportation_wide_df['Total'] = transportation_wide_df.sum(axis=1)

# Translate columns to English
transportation_column_mapping = {
    'Pie': 'On Foot',
    'Bicicleta': 'Bicycle',
    'Caballo': 'Horse',
    'Silla de ruedas': 'Wheelchair',
    'Sin establecer': 'Not specified',
    'Vela': 'Sailboat',
}
transportation_wide_df.rename(columns=transportation_column_mapping, inplace=True)

# Display the resulting DataFrame
dataframes['Transportation'] = transportation_wide_df
del transportation_wide_df


__For df Motivos__

In [44]:
# Directly create the pivot table with summation
motives_wide_df = pd.pivot_table(
    dataframes['Motivos'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',  # Summing the total values
    fill_value=0    # Replacing NaN values with 0 directly
)

# Remove the original DataFrame from the dictionary
dataframes.pop('Motivos', None)

# Calculate the 'Total' column as the sum of the rows
motives_wide_df['Total'] = motives_wide_df.sum(axis=1)

# Translate columns to English
motives_column_mapping = {
    'Religioso': 'Religious',
    'Religioso y otros': 'Religious and others',
    'No religioso': 'Non-religious',
    'Sin establecer' : 'Not specified'
}
motives_wide_df.rename(columns=motives_column_mapping, inplace=True)

  # Use None to avoid KeyError if the key is not found

# Assign the resulting DataFrame back to the dataframes dictionary
dataframes['Motives'] = motives_wide_df
del motives_wide_df


__For df Paises__

In [45]:
# Directly create the pivot table with summation
countries_wide_df = pd.pivot_table(
    dataframes['Paises'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',  # Summing the total values
    fill_value=0    # Replacing NaN values with 0 directly
)

# Remove the original DataFrame from the dictionary
dataframes.pop('Paises', None)

# Calculate the 'Total' column as the sum of the rows
countries_wide_df['Total'] = countries_wide_df.sum(axis=1)

# Translate columns to English
countries_column_mapping = {
    'España': 'Spain',
    'Alemania': 'Germany',
    'Estados Unidos': 'United States',
    'Italia': 'Italy',
    'Australia': 'Australia',
    'Francia': 'France',
    'Suiza': 'Switzerland',
    'Japón': 'Japan',
    'Reino Unido': 'United Kingdom',
    'Holanda': 'Netherlands',
    'Portugal': 'Portugal',
    'Finlandia': 'Finland',
    'Brasil': 'Brazil',
    'México': 'Mexico',
    'Resto': 'Others',
    'Argentina': 'Argentina',
    'Austria': 'Austria',
    'Belgica': 'Belgium',
    'Belice': 'Belize',
    'Bulgaria': 'Bulgaria',
    'Bélgica': 'Belgium',
    'Canada': 'Canada',
    'Chile': 'Chile',
    'China': 'China',
    'Colombia': 'Colombia',
    'Korea': 'Korea',
    'Denmark': 'Denmark',
    'Eslovaquia': 'Slovakia',
    'Estonia': 'Estonia',
    'Hungría': 'Hungary',
    'Indonesia': 'Indonesia',
    'Ireland': 'Ireland',
    'Irán': 'Iran',
    'Islandia': 'Iceland',
    'Israel': 'Israel',
    'Lituania': 'Lithuania',
    'Perú': 'Peru',
    'Poland': 'Poland',
    'Puerto Rico': 'Puerto Rico',
    'Czech Republic': 'Czech Republic',
    'Rumania': 'Romania',
    'Russia': 'Russia',
    'Singapur': 'Singapore',
    'Sudáfrica': 'South Africa',
    'Suecia': 'Sweden',
    'Taiwan': 'Taiwan',
    'Ucrania': 'Ukraine',
    'Uruguay': 'Uruguay',
    'Venezuela': 'Venezuela'
}

countries_wide_df.rename(columns=countries_column_mapping, inplace=True)

# Assign the resulting DataFrame back to the dataframes dictionary
dataframes['Countries'] = countries_wide_df
del countries_wide_df


__For df Procedencias__

In [46]:
# Directly create the pivot table with summation
origin_wide_df = pd.pivot_table(
    dataframes['Procedencias'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',
    fill_value=0
)

# Remove the original DataFrame from the dictionary
dataframes.pop('Procedencias', None)

# Calculate the 'Total' column as the sum of the rows
origin_wide_df['Total'] = origin_wide_df.sum(axis=1)

# Assign the resulting DataFrame back to the dataframes dictionary
dataframes['Origin'] = origin_wide_df
del origin_wide_df


__For df Sexos__

In [47]:
# Directly create the pivot table with summation
gender_wide_df = pd.pivot_table(
    dataframes['Sexos'],
    values='Total',
    index=['ID', 'Year', 'Month'],
    columns='Description',
    aggfunc='sum',
    fill_value=0
)

# Remove the original DataFrame from the dictionary
dataframes.pop('Sexos', None)

# Calculate the 'Total' column as the sum of the rows
gender_wide_df['Total'] = gender_wide_df.sum(axis=1)

# Translate columns to English
gender_column_mapping = {
    'Hombre': 'Male',
    'Mujer': 'Female'
}
gender_wide_df.rename(columns=gender_column_mapping, inplace=True)

# Assign the resulting DataFrame back to the dataframes dictionary
dataframes['Gender'] = gender_wide_df[['Male', 'Female', 'Total']]
del gender_wide_df


__fix the total in Totals__

In [48]:
dataframes['Totals']['Total'] = dataframes['Ages']['Total']


#### __Step 8:__ Output Excel files

In [57]:
# check if path exists and create it if not
if not os.path.exists(save_path_xl):
    os.makedirs(save_path_xl)


In [62]:
import os

if not os.path.exists(save_path_xl):
    os.makedirs(save_path_xl)

with pd.ExcelWriter(f"{save_path_xl}{filename_xlsx}", engine='openpyxl') as writer:
    for key in dataframes.keys():
        dataframes[key].to_excel(writer, sheet_name=key)


#### __Step 9:__ Output CSV files

In [52]:
# check if path exists and create it if not
if not os.path.exists(save_path_csv):
    os.makedirs(save_path_csv)

for key, df in dataframes.items():
    filename_csv = f"{key}.csv"
    full_path = f"{save_path_csv}{filename_csv}"
    df.to_csv(full_path)
